In [2]:
import tkinter as tk
import sounddevice as sd
import numpy as np
import threading

In [3]:
class SoundRecorder:
    def __init__(self):
        self.recording = False
        self.frames = []

    def start_recording(self):
        self.recording = True
        self.frames = []
        threading.Thread(target=self._record_audio).start()

    def stop_recording(self):
        self.recording = False

    def _record_audio(self):
        with sd.InputStream(callback=self._audio_callback):
            sd.sleep(int(5 * 1000))  # Record for 5 seconds

    def _audio_callback(self, indata, frames, time, status):
        if self.recording:
            self.frames.append(indata.copy())

class ControlPanel:
    def __init__(self, root):
        self.root = root
        self.root.title("Sound Recording Control Panel")

        self.recorder = SoundRecorder()

        self.label = tk.Label(root, text="Press space to start recording sound...")
        self.label.pack(pady=20)

        self.root.bind('<KeyPress>', self._on_key_press)

    def _on_key_press(self, event):
        if event.keysym == 'space':
            if not self.recorder.recording:
                self.label.config(text="Recording sound...")
                self.recorder.start_recording()
            else:
                self.label.config(text="Recording stopped")
                self.recorder.stop_recording()
                self._save_audio()

    def _save_audio(self):
        if len(self.recorder.frames) > 0:
            audio_data = np.concatenate(self.recorder.frames, axis=0)
            sd.play(audio_data, self.recorder.frames[0].shape[1])
            sd.wait()
            self.label.config(text="Recording saved and played")

def main():
    root = tk.Tk()
    app = ControlPanel(root)
    root.mainloop()

if __name__ == "__main__":
    main()


In [3]:
import torch
from transformers import pipeline
from transformers.pipelines.audio_utils import ffmpeg_read
import gradio as gr


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: C:\Users\benve\anaconda3\envs\py39\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary C:\Users\benve\anaconda3\envs\py39\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...


In [1]:


MODEL_NAME = "openai/whisper-small"
BATCH_SIZE = 8

device = 0 if torch.cuda.is_available() else "cpu"

pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)

NameError: name 'torch' is not defined

In [6]:
# Copied from https://github.com/openai/whisper/blob/c09a7ae299c4c34c5839a76380ae407e7d785914/whisper/utils.py#L50
def format_timestamp(seconds: float, always_include_hours: bool = False, decimal_marker: str = "."):
    if seconds is not None:
        milliseconds = round(seconds * 1000.0)

        hours = milliseconds // 3_600_000
        milliseconds -= hours * 3_600_000

        minutes = milliseconds // 60_000
        milliseconds -= minutes * 60_000

        seconds = milliseconds // 1_000
        milliseconds -= seconds * 1_000

        hours_marker = f"{hours:02d}:" if always_include_hours or hours > 0 else ""
        return f"{hours_marker}{minutes:02d}:{seconds:02d}{decimal_marker}{milliseconds:03d}"
    else:
        # we have a malformed timestamp so just return it as is
        return seconds


def transcribe(file, task, return_timestamps):
    outputs = pipe(file, batch_size=BATCH_SIZE, generate_kwargs={"task": task}, return_timestamps=return_timestamps)
    text = outputs["text"]
    if return_timestamps:
        timestamps = outputs["chunks"]
        timestamps = [
            f"[{format_timestamp(chunk['timestamp'][0])} -> {format_timestamp(chunk['timestamp'][1])}] {chunk['text']}"
            for chunk in timestamps
        ]
        text = "\n".join(str(feature) for feature in timestamps)
    return text


demo = gr.Blocks()

mic_transcribe = gr.Interface(
    fn=transcribe,
    inputs=[
        gr.inputs.Audio(source="microphone", type="filepath", optional=True),
        gr.inputs.Radio(["transcribe", "translate"], label="Task", default="transcribe"),
        gr.inputs.Checkbox(default=False, label="Return timestamps"),
    ],
    outputs="text",
    layout="horizontal",
    theme="huggingface",
    title="Whisper Demo: Transcribe Audio",
    description=(
        "Transcribe long-form microphone or audio inputs with the click of a button! Demo uses the"
        f" checkpoint [{MODEL_NAME}](https://huggingface.co/{MODEL_NAME}) and 🤗 Transformers to transcribe audio files"
        " of arbitrary length."
    ),
    allow_flagging="never",
)

file_transcribe = gr.Interface(
    fn=transcribe,
    inputs=[
        gr.inputs.Audio(source="upload", optional=True, label="Audio file", type="filepath"),
        gr.inputs.Radio(["transcribe", "translate"], label="Task", default="transcribe"),
        gr.inputs.Checkbox(default=False, label="Return timestamps"),
    ],
    outputs="text",
    layout="horizontal",
    theme="huggingface",
    title="Whisper Demo: Transcribe Audio",
    description=(
        "Transcribe long-form microphone or audio inputs with the click of a button! Demo uses the"
        f" checkpoint [{MODEL_NAME}](https://huggingface.co/{MODEL_NAME}) and 🤗 Transformers to transcribe audio files"
        " of arbitrary length."
    ),
    # examples=[
    #     ["./example.flac", "transcribe", False],
    #     ["./example.flac", "transcribe", True],
    # ],
    cache_examples=True,
    allow_flagging="never",
)

with demo:
    gr.TabbedInterface([mic_transcribe, file_transcribe], ["Transcribe Microphone", "Transcribe Audio File"])

demo.launch(enable_queue=True)

C:\Users\benve\AppData\Local\Temp\ipykernel_23996\4144516193.py:40: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Audio(source="microphone", type="filepath", optional=True),
C:\Users\benve\AppData\Local\Temp\ipykernel_23996\4144516193.py:40: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Audio(source="microphone", type="filepath", optional=True),
C:\Users\benve\AppData\Local\Temp\ipykernel_23996\4144516193.py:41: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Radio(["transcribe", "translate"], label="Task", default="transcribe"),
C:\Users\benve\AppData\Local\Temp\ipykernel_23996\4144516193.py:41: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Radio(["

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


C:\Users\benve\anaconda3\envs\py39\lib\site-packages\gradio\processing_utils.py:188: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
C:\Users\benve\anaconda3\envs\py39\lib\site-packages\transformers\generation\utils.py:1369: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
